In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn
import re

In [2]:
## Input/Output
# DATA_FOLDER = "C:\\Users\\E2884\\Desktop\\WPy64-3980\\notebooks\\data\\SPC\\"
DATA_FOLDER = "SPC_TRADE_FOOD.csv"
FILE = "SPC_FOOD_SECURITY.csv"

In [3]:
df_raw = pd.read_csv(DATA_FOLDER + FILE)
#print(df_raw.head())

Affichage des différentes colonnes et des valeurs pour faire un premier tri et retirer les colonnes inutiles du df --> output df_net

In [4]:
# Export dans excel
# df_net.to_excel(DATA_FOLDER + "SPC_FOOD_SECURITY.xlsx")

In [5]:
# HEADER = list(df_raw.columns)
# for el in HEADER:
#     print(el)
#     print(df_raw[el].unique())

df_net = df_raw.drop(columns=['DATAFLOW', 'FREQ: Frequency','UNIT_MULT: Unit multiplier',\
                              'OBS_STATUS: Observation Status','DATA_SOURCE: Data source',\
                             'OBS_COMMENT: Comment','CONF_STATUS: Confidentiality status'])

HEADER = list(df_net.columns)
for el in HEADER:
    print(el)
    print(df_net[el].unique())

TIME_PERIOD: Time
[2019 2012]
GEO_PICT: Pacific Island Countries and territories
['KI: Kiribati' 'SB: Solomon Islands' 'KI-C: Central Kiribati'
 'KI-A: Northern Kiribati' 'KI-B: South Tarawa' 'KI-E: Line & Phoenix'
 'SB-1: Choiseul' 'SB-9: Temotu' 'SB-5: Rennell-Bell' 'SB-10: Honiara'
 'SB-4: Central' 'SB-2: Western' 'SB-3: Isabel' 'SB-6: Guadalcanal'
 'SB-7: Malaita' 'VU: Vanuatu' 'VU-6: Tafea' 'VU-3: Penama'
 'VU-4: Malampa' 'VU-5: Shefa' 'VU-2: Sanma' 'VU-1: Torba'
 'KI-D: Southern Kiribati' 'SB-8: Makira-Ulawa']
INDICATOR: Indicator
['CONSGTAVG: Percentage of households who consumed more than the average number of products consumed in the group'
 'AQAA: Average quantity as acquired' 'AVGEQ: Average edible quantity'
 'AVGDEC: Average Dietary Energy Consumption'
 'AVGEXP: Average expenditures'
 'CONSAL1: Percentage of HH who consumed at least one product of the group'
 'CONSLTAVG: Percentage of households who consumed less than the average number of products consumed in the group'
 '

In [6]:
df_net = df_net.rename(columns={'TIME_PERIOD: Time': 'YEAR', 
                                'GEO_PICT: Pacific Island Countries and territories': 'REGION',
                                'INDICATOR: Indicator':'INDICATEUR',
                                'SEX: Sex':'GENRE',
                                'AGE: Age':'AGE',
                                'URBANIZATION: Urbanization':'URBANISATION',
                                'POVERTY: Poverty':'POVERTY',
                                'FOOD_SECURITY: Food security':'FOOD_SECURITY',
                                'FOOD: Food category':'FOOD_CATEGORY',
                                'OBS_VALUE':'VALUE',
                                'UNIT_MEASURE: Unit of measure':'UNIT'
                               })
# print(df_net.columns)
# print(df_net)

# Correction de la devise
dict_DEVISE = {'KI': 0.65,'SB': 0.1193 ,'VU': 0.008423}

new_column = []
for reg in df_net['REGION']:
    new_column.append(re.findall("(\w\w).*", reg)[0])
df_net['PAYS'] =  new_column

df_net['PRIX_$'] = df_net['VALUE']*df_net['PAYS'].map(dict_DEVISE)

**Précisions sur les indicateurs :**
 - 'CONSGTAVG (%): Pourcentage de ménages ayant consommé plus que le nombre moyen de produits consommés dans le groupe'
 - 'AQAA (g/capita/day): Quantité moyenne telle qu'acquise'
 - 'AVGEQ (g/capita/day): Quantité comestible moyenne'
 - 'AVGDEC (kcal/capita/day) : Consommation énergétique alimentaire moyenne'
 - 'AVGEXP (dollar/capita/day) : Dépenses moyennes'
 - 'CONSAL1 (%): Pourcentage de ménages ayant consommé au moins un produit du groupe'
 - 'CONSLTAVG: Pourcentage de ménages ayant consommé moins que le nombre moyen de produits consommés dans le groupe'
 - 'AVGNPRDCONS: Nombre moyen de produits consommés par ménage par groupe alimentaire'
 
 CONSGTAVG + CONSLTAVG = CONSAL1
 
 1 - CONSAL1 = % des ménages n'ayant consommés aucun produit dans le groupe
 
**Les filtres :** 
 - YEAR : 2012 et 2019
 - REGION : 
     - ['KI: Kiribati','KI-A: Northern Kiribati','KI-B: South Tarawa','KI-C: Central Kiribati','KI-D: Southern Kiribati','KI-E: Line & Phoenix']
     - ['SB: Solomon Islands', 'SB-1: Choiseul','SB-2:    Western','SB-3: Isabel' ,'SB-4: Central','SB-5: Rennell-Bell','SB-6: Guadalcanal','SB-7: Malaita','SB-8: Makira-Ulawa','SB-9: Temotu', 'SB-10: Honiara'] 
     - ['VU: Vanuatu': 'VU-1: Torba','VU-2: Sanma','VU-3: Penama','VU-4: Malampa','VU-5: Shefa','VU-6: Tafea']
 - GENRE : 
     - 'F: Female'
     - 'M: Male'
     - '_T: Total'
 - AGE : 
     - '_T: All ages'
     - 'Y15T24: 15-24'
     - 'Y25T59: 25-59'
     - 'Y60T999: 60+'
 - URBANISATION : 
     - '_T: National'
     - 'U: Urban'
     - 'R: Rural'
 - POVERTY : 
     - '_T: Total'
     - 'NOP: No poverty'
     - 'POV: Poverty'
 - FOOD_SECURITY : 
     - '_T: Total'
     - 'FOODSEC: Food secure or mildly food insecur'
     - 'FOODINS: Moderately or severely food insecure'
     
Pour le vanuatu et Kiribati, il y a seulement l'année 2019, pour Solomon Island l'année 2012

## Par pays
- Valeur totale $
- Valeur total kcal/capita/day
- Valeur total kcal/capita/day (edible)
- Valeur total kcal/capita/day (aquired)
- Waste

puis filter par tranche d'age/genre/urbanisation/poverty/food insecure

In [8]:
# Par age
region_list = ['KI: Kiribati','SB: Solomon Islands','VU: Vanuatu']
df_pays_age = df_net[df_net['REGION'].isin(region_list)]
df_pays_age = df_pays_age.loc[(df_pays_age['GENRE'] == '_T: Total') & \
#                           (df_pays_age['AGE'] == '_T: All ages') & \
                          (df_pays_age['URBANISATION'] == '_T: National') & \
                          (df_pays_age['POVERTY'] == '_T: Total') & \
                          (df_pays_age['FOOD_SECURITY'] == '_T: Total')
                         ]
df_pays_age_AVGEXP = df_pays_age[df_pays_age['INDICATEUR'] == 'AVGEXP: Average expenditures']
df_gb_pays_age_AVGEXP = df_pays_age_AVGEXP.groupby(['REGION','AGE']).agg({'PRIX_$':'sum'}).rename(columns={"PRIX_$":"VALEUR_$"})
# print("$ par jour et par personne pour un repas")
# print(df_gb_pays_ageAVGEXP)
# print("\n")

df_pays_age_AVGDEC = df_pays_age[df_pays_age['INDICATEUR'] == 'AVGDEC: Average Dietary Energy Consumption']
df_gb_pays_age_AVGDEC = df_pays_age_AVGDEC.groupby(['REGION','AGE']).agg({'VALUE':'sum'}).rename(columns={"VALUE":"VALEUR_KCAL"})
# print("kcal par jour et par personne pour un repas")
# print(df_gb_pays_ageAVGDEC)
# print("\n")

df_pays_age_AVGEQ = df_pays_age[df_pays_age['INDICATEUR'] == 'AVGEQ: Average edible quantity']
df_gb_pays_age_AVGEQ = df_pays_age_AVGEQ.groupby(['REGION','AGE']).agg({'VALUE':'sum'}).rename(columns={"VALUE":"VALEUR_G edible"})
# print("g edible par jour et par personne pour un repas")
# print(df_gb_pays_ageAVGEQ)
# print("\n")

df_pays_age_AQAA = df_pays_age[df_pays_age['INDICATEUR'] == 'AQAA: Average quantity as acquired']
df_gb_pays_age_AQAA = df_pays_age_AQAA.groupby(['REGION','AGE']).agg({'VALUE':'sum'}).rename(columns={"VALUE":"VALEUR_G aquired"})
# print("g aquired par jour et par personne pour un repas")
# print(df_gb_pays_ageAQAA)
# print("\n")

df_gb_pays_age_all = pd.concat([df_gb_pays_age_AVGEXP, df_gb_pays_age_AVGDEC,df_gb_pays_age_AVGEQ,df_gb_pays_age_AQAA], axis=1)
df_gb_pays_age_all['WASTE'] = df_gb_pays_age_all['VALEUR_G aquired']-df_gb_pays_age_all['VALEUR_G edible']
print(df_gb_pays_age_all)

                                   VALEUR_$  VALEUR_KCAL  VALEUR_G edible  \
REGION              AGE                                                     
KI: Kiribati        Y15T24: 15-24  2.340000       2645.2           1267.0   
                    Y25T59: 25-59  2.275000       2804.0           1231.0   
                    Y60T999: 60+   1.950000       2517.4           1090.0   
                    _T: All ages   2.275000       2757.1           1210.4   
SB: Solomon Islands Y15T24: 15-24  2.087750       3160.1           1970.5   
                    Y25T59: 25-59  1.705990       2595.1           1519.0   
                    Y60T999: 60+   1.682130       2894.1           1719.4   
                    _T: All ages   1.705990       2640.6           1550.8   
VU: Vanuatu         Y15T24: 15-24  3.794561       3017.5           2210.6   
                    Y25T59: 25-59  3.365831       2716.1           1888.7   
                    Y60T999: 60+   3.247909       2922.0           2023.4   

- PIB VU = 956.3 M$ / POP = 319137 et PIB/HAB = 2996 $
- PIB SB = 1.631 M$ / POP = 707851 et PIB/HAB = 2300 $ 
- PIB KI = 207 M$ / POP = 128874 et PIB/HAB = 1606 $

Dans les 3 régions, plus on veillit, moins le panier coute cher
La valeur nutrive par panier diffère par tranche d'age et par pays ? A Creuser au niveau des catégories de nourriture.
Beaucoup moins de déchet pour Ki que pour les deux autres ? A Creuser au niveau des catégories de nourriture.

In [9]:
# Par genre
region_list = ['KI: Kiribati','SB: Solomon Islands','VU: Vanuatu']
df_pays_genre = df_net[df_net['REGION'].isin(region_list)]
df_pays_genre = df_pays_genre.loc[
#                           (df_pays_genre['GENRE'] == '_T: Total') & \
                          (df_pays_genre['AGE'] == '_T: All ages') & \
                          (df_pays_genre['URBANISATION'] == '_T: National') & \
                          (df_pays_genre['POVERTY'] == '_T: Total') & \
                          (df_pays_genre['FOOD_SECURITY'] == '_T: Total')
                         ]
df_pays_genre_AVGEXP = df_pays_genre[df_pays_genre['INDICATEUR'] == 'AVGEXP: Average expenditures']
df_gb_pays_genre_AVGEXP = df_pays_genre_AVGEXP.groupby(['REGION','GENRE']).agg({'PRIX_$':'sum'}).rename(columns={"PRIX_$":"VALEUR_$"})
# print("$ par jour et par personne pour un repas")
# print(df_gb_AVGEXP)
# print("\n")

df_pays_genre_AVGDEC = df_pays_genre[df_pays_genre['INDICATEUR'] == 'AVGDEC: Average Dietary Energy Consumption']
df_gb_pays_genre_AVGDEC = df_pays_genre_AVGDEC.groupby(['REGION','GENRE']).agg({'VALUE':'sum'}).rename(columns={"VALUE":"VALEUR_KCAL"})
# print("kcal par jour et par personne pour un repas")
# print(df_gb_AVGDEC)
# print("\n")

df_pays_genre_AVGEQ = df_pays_genre[df_pays_genre['INDICATEUR'] == 'AVGEQ: Average edible quantity']
df_gb_pays_genre_AVGEQ = df_pays_genre_AVGEQ.groupby(['REGION','GENRE']).agg({'VALUE':'sum'}).rename(columns={"VALUE":"VALEUR_G edible"})
# print("g edible par jour et par personne pour un repas")
# print(df_gb_AVGEQ)
# print("\n")

df_pays_genre_AQAA = df_pays_genre[df_pays_genre['INDICATEUR'] == 'AQAA: Average quantity as acquired']
df_gb_pays_genre_AQAA = df_pays_genre_AQAA.groupby(['REGION','GENRE']).agg({'VALUE':'sum'}).rename(columns={"VALUE":"VALEUR_G aquired"})
# print("g aquired par jour et par personne pour un repas")
# print(df_gb_AQAA)
# print("\n")

df_gb_pays_genre_all = pd.concat([df_gb_pays_genre_AVGEXP, df_gb_pays_genre_AVGDEC,df_gb_pays_genre_AVGEQ,df_gb_pays_genre_AQAA], axis=1)
df_gb_pays_genre_all['WASTE'] = df_gb_pays_genre_all['VALEUR_G aquired']-df_gb_pays_genre_all['VALEUR_G edible']
print(df_gb_pays_genre_all)

                               VALEUR_$  VALEUR_KCAL  VALEUR_G edible  \
REGION              GENRE                                               
KI: Kiribati        F: Female  2.210000       2740.5           1160.9   
                    M: Male    2.275000       2765.7           1234.5   
                    _T: Total  2.275000       2757.1           1210.4   
SB: Solomon Islands F: Female  1.741780       2814.0           1625.3   
                    M: Male    1.682130       2624.5           1544.0   
                    _T: Total  1.705990       2640.6           1550.8   
VU: Vanuatu         F: Female  3.590725       2810.9           1904.4   
                    M: Male    3.238643       2727.8           1932.2   
                    _T: Total  3.367515       2758.3           1922.1   

                               VALEUR_G aquired  WASTE  
REGION              GENRE                               
KI: Kiribati        F: Female            1337.1  176.2  
                    M: Ma

Relativement similaire au niveau des disparités H/F sur tous les aspects du tableau

In [10]:
# Par URBANISATION
region_list = ['KI: Kiribati','SB: Solomon Islands','VU: Vanuatu']
df_pays_urb = df_net[df_net['REGION'].isin(region_list)]
df_pays_urb = df_pays_urb.loc[
                          (df_pays_urb['GENRE'] == '_T: Total') & \
                          (df_pays_urb['AGE'] == '_T: All ages') & \
#                           (df_pays_urb['URBANISATION'] == '_T: National') & \
                          (df_pays_urb['POVERTY'] == '_T: Total') & \
                          (df_pays_urb['FOOD_SECURITY'] == '_T: Total')
                         ]
df_pays_urb_AVGEXP = df_pays_urb[df_pays_urb['INDICATEUR'] == 'AVGEXP: Average expenditures']
df_gb_pays_urb_AVGEXP = df_pays_urb_AVGEXP.groupby(['REGION','URBANISATION']).agg({'PRIX_$':'sum'}).rename(columns={"PRIX_$":"VALEUR_$"})
# print("$ par jour et par personne pour un repas")
# print(df_gb_AVGEXP)
# print("\n")

df_pays_urb_AVGDEC = df_pays_urb[df_pays_urb['INDICATEUR'] == 'AVGDEC: Average Dietary Energy Consumption']
df_gb_pays_urb_AVGDEC = df_pays_urb_AVGDEC.groupby(['REGION','URBANISATION']).agg({'VALUE':'sum'}).rename(columns={"VALUE":"VALEUR_KCAL"})
# print("kcal par jour et par personne pour un repas")
# print(df_gb_AVGDEC)
# print("\n")

df_pays_urb_AVGEQ = df_pays_urb[df_pays_urb['INDICATEUR'] == 'AVGEQ: Average edible quantity']
df_gb_pays_urb_AVGEQ = df_pays_urb_AVGEQ.groupby(['REGION','URBANISATION']).agg({'VALUE':'sum'}).rename(columns={"VALUE":"VALEUR_G edible"})
# print("g edible par jour et par personne pour un repas")
# print(df_gb_AVGEQ)
# print("\n")

df_pays_urb_AQAA = df_pays_urb[df_pays_urb['INDICATEUR'] == 'AQAA: Average quantity as acquired']
df_gb_pays_urb_AQAA = df_pays_urb_AQAA.groupby(['REGION','URBANISATION']).agg({'VALUE':'sum'}).rename(columns={"VALUE":"VALEUR_G aquired"})
# print("g aquired par jour et par personne pour un repas")
# print(df_gb_AQAA)
# print("\n")

df_gb_pays_urb_all = pd.concat([df_gb_pays_urb_AVGEXP, df_gb_pays_urb_AVGDEC,df_gb_pays_urb_AVGEQ,df_gb_pays_urb_AQAA], axis=1)
df_gb_pays_urb_all['WASTE'] = df_gb_pays_urb_all['VALEUR_G aquired']-df_gb_pays_urb_all['VALEUR_G edible']
print(df_gb_pays_urb_all)

                                  VALEUR_$  VALEUR_KCAL  VALEUR_G edible  \
REGION              URBANISATION                                           
KI: Kiribati        R: Rural      2.145000       2606.5           1221.9   
                    U: Urban      2.145000       2889.4           1200.1   
                    _T: National  2.275000       2757.1           1210.4   
SB: Solomon Islands R: Rural      1.598620       2738.6           1674.3   
                    U: Urban      2.135470       2211.6           1008.7   
                    _T: National  1.705990       2640.6           1550.8   
VU: Vanuatu         R: Rural      2.962369       2663.6           1911.1   
                    U: Urban      4.509674       3023.5           1953.2   
                    _T: National  3.367515       2758.3           1922.1   

                                  VALEUR_G aquired  WASTE  
REGION              URBANISATION                           
KI: Kiribati        R: Rural               

KI : assez homogène 
SB : assez hétéorogène notamment prix/kcal
VU : assez hétérogène mais dans l'autre sens du SB

In [11]:
# Par POVERTY
region_list = ['KI: Kiribati','SB: Solomon Islands','VU: Vanuatu']
df_pays_poverty = df_net[df_net['REGION'].isin(region_list)]
df_pays_poverty = df_pays_poverty.loc[
                          (df_pays_poverty['GENRE'] == '_T: Total') & \
                          (df_pays_poverty['AGE'] == '_T: All ages') & \
                          (df_pays_poverty['URBANISATION'] == '_T: National') & \
#                           (df_pays_poverty['POVERTY'] == '_T: Total') & \
                          (df_pays_poverty['FOOD_SECURITY'] == '_T: Total')
                         ]
df_pays_poverty_AVGEXP = df_pays_poverty[df_pays_poverty['INDICATEUR'] == 'AVGEXP: Average expenditures']
df_gb_pays_poverty_AVGEXP = df_pays_poverty_AVGEXP.groupby(['REGION','POVERTY']).agg({'PRIX_$':'sum'}).rename(columns={"PRIX_$":"VALEUR_$"})
# print("$ par jour et par personne pour un repas")
# print(df_gb_AVGEXP)
# print("\n")

df_pays_poverty_AVGDEC = df_pays_poverty[df_pays_poverty['INDICATEUR'] == 'AVGDEC: Average Dietary Energy Consumption']
df_gb_pays_poverty_AVGDEC = df_pays_poverty_AVGDEC.groupby(['REGION','POVERTY']).agg({'VALUE':'sum'}).rename(columns={"VALUE":"VALEUR_KCAL"})
# print("kcal par jour et par personne pour un repas")
# print(df_gb_AVGDEC)
# print("\n")

df_pays_poverty_AVGEQ = df_pays_poverty[df_pays_poverty['INDICATEUR'] == 'AVGEQ: Average edible quantity']
df_gb_pays_poverty_AVGEQ = df_pays_poverty_AVGEQ.groupby(['REGION','POVERTY']).agg({'VALUE':'sum'}).rename(columns={"VALUE":"VALEUR_G edible"})
# print("g edible par jour et par personne pour un repas")
# print(df_gb_AVGEQ)
# print("\n")

df_pays_poverty_AQAA = df_pays_poverty[df_pays_poverty['INDICATEUR'] == 'AQAA: Average quantity as acquired']
df_gb_pays_poverty_AQAA = df_pays_poverty_AQAA.groupby(['REGION','POVERTY']).agg({'VALUE':'sum'}).rename(columns={"VALUE":"VALEUR_G aquired"})
# print("g aquired par jour et par personne pour un repas")
# print(df_gb_AQAA)
# print("\n")

df_gb_pays_poverty_all = pd.concat([df_gb_pays_poverty_AVGEXP, df_gb_pays_poverty_AVGDEC,df_gb_pays_poverty_AVGEQ,df_gb_pays_poverty_AQAA], axis=1)
df_gb_pays_poverty_all['WASTE'] = df_gb_pays_poverty_all['VALEUR_G aquired']-df_gb_pays_poverty_all['VALEUR_G edible']
print(df_gb_pays_poverty_all)

                                     VALEUR_$  VALEUR_KCAL  VALEUR_G edible  \
REGION              POVERTY                                                   
KI: Kiribati        NOP: No poverty  2.600000       2988.8           1324.0   
                    POV: Poverty     1.105000       1931.7            805.0   
                    _T: Total        2.275000       2757.1           1210.4   
SB: Solomon Islands NOP: No poverty  1.837220       2753.8           1603.8   
                    POV: Poverty     0.823170       1866.7           1187.7   
                    _T: Total        1.705990       2640.6           1550.8   
VU: Vanuatu         NOP: No poverty  3.637051       2856.2           1972.9   
                    POV: Poverty     1.149740       1369.7           1007.9   
                    _T: Total        3.367515       2758.3           1922.1   

                                     VALEUR_G aquired  WASTE  
REGION              POVERTY                                   
KI: 

Dans les 3 cas les pauvres mangent moins que les riches en valeur et en kcal par pannier / il y a logiquement moins de gaspillage

In [12]:
# Par FOOD_SECURITY
region_list = ['KI: Kiribati','SB: Solomon Islands','VU: Vanuatu']
df_pays_fs = df_net[df_net['REGION'].isin(region_list)]
df_pays_fs = df_pays_fs.loc[
                          (df_pays_fs['GENRE'] == '_T: Total') & \
                          (df_pays_fs['AGE'] == '_T: All ages') & \
                          (df_pays_fs['URBANISATION'] == '_T: National') & \
                          (df_pays_fs['POVERTY'] == '_T: Total')
#                           (df_pays_fs['FOOD_SECURITY'] == '_T: Total')
                         ]
df_pays_fs_AVGEXP = df_pays_fs[df_pays_fs['INDICATEUR'] == 'AVGEXP: Average expenditures']
df_gb_pays_fs_AVGEXP = df_pays_fs_AVGEXP.groupby(['REGION','FOOD_SECURITY']).agg({'PRIX_$':'sum'}).rename(columns={"PRIX_$":"VALEUR_$"})
# print("$ par jour et par personne pour un repas")
# print(df_gb_AVGEXP)
# print("\n")

df_pays_fs_AVGDEC = df_pays_fs[df_pays_fs['INDICATEUR'] == 'AVGDEC: Average Dietary Energy Consumption']
df_gb_pays_fs_AVGDEC = df_pays_fs_AVGDEC.groupby(['REGION','FOOD_SECURITY']).agg({'VALUE':'sum'}).rename(columns={"VALUE":"VALEUR_KCAL"})
# print("kcal par jour et par personne pour un repas")
# print(df_gb_AVGDEC)
# print("\n")

df_pays_fs_AVGEQ = df_pays_fs[df_pays_fs['INDICATEUR'] == 'AVGEQ: Average edible quantity']
df_gb_pays_fs_AVGEQ = df_pays_fs_AVGEQ.groupby(['REGION','FOOD_SECURITY']).agg({'VALUE':'sum'}).rename(columns={"VALUE":"VALEUR_G edible"})
# print("g edible par jour et par personne pour un repas")
# print(df_gb_AVGEQ)
# print("\n")

df_pays_fs_AQAA = df_pays_fs[df_pays_fs['INDICATEUR'] == 'AQAA: Average quantity as acquired']
df_gb_pays_fs_AQAA = df_pays_fs_AQAA.groupby(['REGION','FOOD_SECURITY']).agg({'VALUE':'sum'}).rename(columns={"VALUE":"VALEUR_G aquired"})
# print("g aquired par jour et par personne pour un repas")
# print(df_gb_AQAA)
# print("\n")

df_gb_pays_fs_all = pd.concat([df_gb_pays_fs_AVGEXP, df_gb_pays_fs_AVGDEC,df_gb_pays_fs_AVGEQ,df_gb_pays_fs_AQAA], axis=1)
df_gb_pays_fs_all['WASTE'] = df_gb_pays_fs_all['VALEUR_G aquired']-df_gb_pays_fs_all['VALEUR_G edible']
print(df_gb_pays_fs_all)

                                                                   VALEUR_$  \
REGION              FOOD_SECURITY                                             
KI: Kiribati        FOODINS: Moderately or severely food insecure  2.015000   
                    FOODSEC: Food secure or mildly food insecur    2.405000   
                    _T: Total                                      2.275000   
SB: Solomon Islands _T: Total                                      1.705990   
VU: Vanuatu         FOODINS: Moderately or severely food insecure  3.188948   
                    FOODSEC: Food secure or mildly food insecur    3.428161   
                    _T: Total                                      3.367515   

                                                                   VALEUR_KCAL  \
REGION              FOOD_SECURITY                                                
KI: Kiribati        FOODINS: Moderately or severely food insecure       2628.7   
                    FOODSEC: Food secure o

## Par régions dans un pays

In [18]:
# KIRIBATI
region_list = ['KI: Kiribati','KI-A: Northern Kiribati','KI-B: South Tarawa','KI-C: Central Kiribati','KI-D: Southern Kiribati','KI-E: Line & Phoenix']
df_region_ki = df_net[df_net['REGION'].isin(region_list)]
df_region_ki = df_region_ki.loc[(df_region_ki['GENRE'] == '_T: Total') & \
                          (df_region_ki['AGE'] == '_T: All ages') & \
                          (df_region_ki['URBANISATION'] == '_T: National') & \
                          (df_region_ki['POVERTY'] == '_T: Total') & \
                          (df_region_ki['FOOD_SECURITY'] == '_T: Total')
                         ]
df_region_ki_AVGEXP = df_region_ki[df_region_ki['INDICATEUR'] == 'AVGEXP: Average expenditures']
df_gb_region_ki_AVGEXP = df_region_ki_AVGEXP.groupby(['REGION']).agg({'PRIX_$':'sum'}).rename(columns={"PRIX_$":"VALEUR_$"})
# print("$ par jour et par personne pour un repas")
# print(df_gb_region_ki_AVGEXP)
# print("\n")

df_region_ki_AVGDEC = df_region_ki[df_region_ki['INDICATEUR'] == 'AVGDEC: Average Dietary Energy Consumption']
df_gb_region_ki_AVGDEC = df_region_ki_AVGDEC.groupby(['REGION']).agg({'VALUE':'sum'}).rename(columns={"VALUE":"VALEUR_KCAL"})
# print("kcal par jour et par personne pour un repas")
# print(df_gb_region_ki_AVGDEC)
# print("\n")

df_region_ki_AVGEQ = df_region_ki[df_region_ki['INDICATEUR'] == 'AVGEQ: Average edible quantity']
df_gb_region_ki_AVGEQ = df_region_ki_AVGEQ.groupby(['REGION']).agg({'VALUE':'sum'}).rename(columns={"VALUE":"VALEUR_G edible"})
# print("g edible par jour et par personne pour un repas")
# print(df_gb_region_ki_AVGEQ)
# print("\n")

df_region_ki_AQAA = df_region_ki[df_region_ki['INDICATEUR'] == 'AQAA: Average quantity as acquired']
df_gb_region_ki_AQAA = df_region_ki_AQAA.groupby(['REGION']).agg({'VALUE':'sum'}).rename(columns={"VALUE":"VALEUR_G aquired"})
# print("g aquired par jour et par personne pour un repas")
# print(df_gb_region_ki_AQAA)
# print("\n")

df_gb_region_ki_all = pd.concat([df_gb_region_ki_AVGEXP, df_gb_region_ki_AVGDEC,df_gb_region_ki_AVGEQ,df_gb_region_ki_AQAA], axis=1)
df_gb_region_ki_all['WASTE'] = df_gb_region_ki_all['VALEUR_G aquired']-df_gb_region_ki_all['VALEUR_G edible']
print(df_gb_region_ki_all)

                         VALEUR_$  VALEUR_KCAL  VALEUR_G edible  \
REGION                                                            
KI-A: Northern Kiribati     1.950       2636.3           1181.3   
KI-B: South Tarawa          2.145       2889.4           1200.1   
KI-C: Central Kiribati      2.405       2932.5           1376.5   
KI-D: Southern Kiribati     1.755       2409.0           1190.4   
KI-E: Line & Phoenix        2.470       2574.9           1222.7   
KI: Kiribati                2.275       2757.1           1210.4   

                         VALEUR_G aquired  WASTE  
REGION                                            
KI-A: Northern Kiribati            1439.5  258.2  
KI-B: South Tarawa                 1361.0  160.9  
KI-C: Central Kiribati             1677.4  300.9  
KI-D: Southern Kiribati            1413.2  222.8  
KI-E: Line & Phoenix               1554.0  331.3  
KI: Kiribati                       1420.4  210.0  


In [19]:
# Iles solomon
region_list = ['SB: Solomon Islands', 'SB-1: Choiseul','SB-2:    Western','SB-3: Isabel' ,'SB-4: Central','SB-5: Rennell-Bell','SB-6: Guadalcanal','SB-7: Malaita','SB-8: Makira-Ulawa','SB-9: Temotu', 'SB-10: Honiara'] 
df_region_sb = df_net[df_net['REGION'].isin(region_list)]
df_region_sb = df_region_sb.loc[(df_region_sb['GENRE'] == '_T: Total') & \
                          (df_region_sb['AGE'] == '_T: All ages') & \
                          (df_region_sb['URBANISATION'] == '_T: National') & \
                          (df_region_sb['POVERTY'] == '_T: Total') & \
                          (df_region_sb['FOOD_SECURITY'] == '_T: Total')
                         ]
df_region_sb_AVGEXP = df_region_sb[df_region_sb['INDICATEUR'] == 'AVGEXP: Average expenditures']
df_gb_region_sb_AVGEXP = df_region_sb_AVGEXP.groupby(['REGION']).agg({'PRIX_$':'sum'}).rename(columns={"PRIX_$":"VALEUR_$"})
# print("$ par jour et par personne pour un repas")
# print(df_gb_region_sb_AVGEXP)
# print("\n")

df_region_sb_AVGDEC = df_region_sb[df_region_sb['INDICATEUR'] == 'AVGDEC: Average Dietary Energy Consumption']
df_gb_region_sb_AVGDEC = df_region_sb_AVGDEC.groupby(['REGION']).agg({'VALUE':'sum'}).rename(columns={"VALUE":"VALEUR_KCAL"})
# print("kcal par jour et par personne pour un repas")
# print(df_gb_region_sb_AVGDEC)
# print("\n")

df_region_sb_AVGEQ = df_region_sb[df_region_sb['INDICATEUR'] == 'AVGEQ: Average edible quantity']
df_gb_region_sb_AVGEQ = df_region_sb_AVGEQ.groupby(['REGION']).agg({'VALUE':'sum'}).rename(columns={"VALUE":"VALEUR_G edible"})
# print("g edible par jour et par personne pour un repas")
# print(df_gb_region_sb_AVGEQ)
# print("\n")

df_region_sb_AQAA = df_region_sb[df_region_sb['INDICATEUR'] == 'AQAA: Average quantity as acquired']
df_gb_region_sb_AQAA = df_region_sb_AQAA.groupby(['REGION']).agg({'VALUE':'sum'}).rename(columns={"VALUE":"VALEUR_G aquired"})
# print("g aquired par jour et par personne pour un repas")
# print(df_gb_region_sb_AQAA)
# print("\n")

df_gb_region_sb_all = pd.concat([df_gb_region_sb_AVGEXP, df_gb_region_sb_AVGDEC,df_gb_region_sb_AVGEQ,df_gb_region_sb_AQAA], axis=1)
df_gb_region_sb_all['WASTE'] = df_gb_region_sb_all['VALEUR_G aquired']-df_gb_region_sb_all['VALEUR_G edible']
print(df_gb_region_sb_all)

                     VALEUR_$  VALEUR_KCAL  VALEUR_G edible  VALEUR_G aquired  \
REGION                                                                          
SB-10: Honiara        2.18319       2138.7            958.1            1136.1   
SB-1: Choiseul        1.37195       2243.2           1462.0            1836.1   
SB-3: Isabel          1.87301       3018.2           1993.2            2498.6   
SB-4: Central         1.80143       3441.0           2168.1            2683.4   
SB-5: Rennell-Bell    2.05196       2662.5           1587.7            2140.1   
SB-6: Guadalcanal     1.71792       2710.9           1459.7            1827.6   
SB-7: Malaita         1.62248       2470.9           1515.9            1915.5   
SB-8: Makira-Ulawa    1.19300       2752.1           1763.3            2352.5   
SB-9: Temotu          1.56283       2799.1           1822.9            2419.3   
SB: Solomon Islands   1.70599       2640.6           1550.8            1956.7   

                     WASTE 

In [20]:
# Vanuatu
region_list = ['VU: Vanuatu','VU-1: Torba','VU-2: Sanma','VU-3: Penama','VU-4: Malampa','VU-5: Shefa','VU-6: Tafea']
df_region_vu = df_net[df_net['REGION'].isin(region_list)]
df_region_vu = df_region_vu.loc[(df_region_vu['GENRE'] == '_T: Total') & \
                          (df_region_vu['AGE'] == '_T: All ages') & \
                          (df_region_vu['URBANISATION'] == '_T: National') & \
                          (df_region_vu['POVERTY'] == '_T: Total') & \
                          (df_region_vu['FOOD_SECURITY'] == '_T: Total')
                         ]
df_region_vu_AVGEXP = df_region_vu[df_region_vu['INDICATEUR'] == 'AVGEXP: Average expenditures']
df_gb_region_vu_AVGEXP = df_region_vu_AVGEXP.groupby(['REGION']).agg({'PRIX_$':'sum'}).rename(columns={"PRIX_$":"VALEUR_$"})
# print("$ par jour et par personne pour un repas")
# print(df_gb_region_vu_AVGEXP)
# print("\n")

df_region_vu_AVGDEC = df_region_vu[df_region_vu['INDICATEUR'] == 'AVGDEC: Average Dietary Energy Consumption']
df_gb_region_vu_AVGDEC = df_region_vu_AVGDEC.groupby(['REGION']).agg({'VALUE':'sum'}).rename(columns={"VALUE":"VALEUR_KCAL"})
# print("kcal par jour et par personne pour un repas")
# print(df_gb_region_vu_AVGDEC)
# print("\n")

df_region_vu_AVGEQ = df_region_vu[df_region_vu['INDICATEUR'] == 'AVGEQ: Average edible quantity']
df_gb_region_vu_AVGEQ = df_region_vu_AVGEQ.groupby(['REGION']).agg({'VALUE':'sum'}).rename(columns={"VALUE":"VALEUR_G edible"})
# print("g edible par jour et par personne pour un repas")
# print(df_gb_region_vu_AVGEQ)
# print("\n")

df_region_vu_AQAA = df_region_vu[df_region_vu['INDICATEUR'] == 'AQAA: Average quantity as acquired']
df_gb_region_vu_AQAA = df_region_vu_AQAA.groupby(['REGION']).agg({'VALUE':'sum'}).rename(columns={"VALUE":"VALEUR_G aquired"})
# print("g aquired par jour et par personne pour un repas")
# print(df_gb_region_vu_AQAA)
# print("\n")

df_gb_region_vu_all = pd.concat([df_gb_region_vu_AVGEXP, df_gb_region_vu_AVGDEC,df_gb_region_vu_AVGEQ,df_gb_region_vu_AQAA], axis=1)
df_gb_region_vu_all['WASTE'] = df_gb_region_vu_all['VALEUR_G aquired']-df_gb_region_vu_all['VALEUR_G edible']
print(df_gb_region_vu_all)

               VALEUR_$  VALEUR_KCAL  VALEUR_G edible  VALEUR_G aquired  WASTE
REGION                                                                        
VU-1: Torba    2.747583       2275.6           1759.1            2358.5  599.4
VU-3: Penama   2.499946       2742.9           1928.0            2527.0  599.0
VU-4: Malampa  3.762554       2801.8           1917.5            2342.6  425.1
VU-6: Tafea    4.716880       3019.0           1985.9            2282.7  296.8
VU: Vanuatu    3.367515       2758.3           1922.1            2372.2  450.1
